# Grouping and Capturing

In this chapter, we cover the final set of metacharacters, the parentheses, `( )`. They are the most complex and powerful special regex characters, providing the ability to group together parts of the regex syntax, capture specific pieces of text, look ahead, and look behind.

In [ ]:
import re
import pandas as pd

movie = pd.read_csv('../data/movie.csv')
title = movie['title']

def find_pattern(s, pattern, **kwargs):
    filt = s.str.contains(pattern, **kwargs)
    return s[filt]

## Grouping with parentheses `( )`

We begin our coverage of the parentheses special characters by learning how they can be used to group together syntax within a single pattern. For instance, let's say we want to find all movies that begin with the word `'In'` or `'My'`. You might think about using the pattern `'^In|My'`. 

In [ ]:
find_pattern(title, r'^In|My').head(8)

### The meaning of `^In|My`

Looking at the result above, the movie `'Journey 2: The Mysterious Island'` does not begin with `'In'` or `'My'` but instead has `'My'` within it. Also, most of the movies don't begin with the word `'In'`, but with a word that starts with it. The pattern `^In|My` matches movies that begin with the letters `'In'` **OR** have `'My'` anywhere inside it. The caret only anchors `'In'` to the start.

## Using parentheses to change operator precedence

We can use parentheses to change the operator precedence just how we do in mathematical expressions. Let's modify our expression to `'^(In|My)\b'`, which groups together the or condition first and then anchors it to the start. We add a word boundary so that the final regex meaning matches our original intent. Ignore the warning for now. We will take care of it below.

In [ ]:
find_pattern(title, r'^(In|My)\b').head()

### Why are we getting `UserWarning: This pattern has match groups`?

Besides operator precedence, parentheses have an alternative purpose, and that is to extract specific text from a string. In regex terminology, we call this a **capturing group**. This warning is alerting us that we have used the syntax for a capture group but are not using a method to do extraction. It tells us to use the `extract` method if we were interested in extracting this group.

### Specifying a non-capturing group

Our regular expression is valid in its current state. We can signal that this is a **non-capturing group** by placing `?:` as the first two characters inside of the parentheses. This eliminates the warning.

In [ ]:
find_pattern(title, r'^(?:In|My)\b').head()

## Using capture groups with the `extract` string method

We can use the exact same pattern with the `extract` string method to extract the group. Here, we use the method directly. The `find_pattern` function will only be used for boolean matching with the `contains` method.

In [ ]:
title.str.extract(r'^(In|My)\b').head()

### Why are all the values missing?

The `extract` method returns a DataFrame with one row for each value in the Series, and one column for each capture group. Since there is only a single capture group, we are returned a one-column DataFrame. Only a small fraction of the movie titles begin with `'In'` or `'My'`, and when the pattern does not match the string, a missing value is returned. Let's drop the missing values and view the extracted text from the titles that do match the pattern.

In [ ]:
title.str.extract(r'^(In|My)\b').dropna().head()

### Return a Series by setting  `expand=False`

When extracting a single capture group set `expand` to `False` to return a Series.

In [ ]:
title.str.extract(r'^(In|My)\b', expand=False).dropna().head()

### Extracting the fourth word of movie titles that begin with 'In' or 'My'

Let's try something a bit more complex and extract the fourth word of those movies that begin with the words `'In'` or `'My'`. For instance, the movie, 'In the Heart of the Sea' meets our criteria. The word 'of' would be extracted from it. We will make the assumption that words are separated by spaces.

To accomplish this, we need to match movies that begin with 'In' or 'My' and then match two words, before capturing the fourth word. We already saw that `^(?:In|My)` completes the first part of this task. 

We now need to match a space followed by a word. We can use `\s` to match a space and follow this with `\S+` to match any number of non-space characters. Combining these, `\s\S+` is what we can use to match our definition of a "word".

We want to match this pattern exactly twice. We can do so with `{2}`, but we must ensure that the it applies to all of `\s\S+`, so we must wrap it in parentheses to control for operator precedence like this - `(\s\S+){2}`. We must signal that this is not a capturing group and arrive at `(?:\s\S+){2}` to match two consecutive words.

We still need to match a space after the third word and then capture the fourth word. Finally, we have a workable regex with the following:

In [ ]:
pattern = r'^(?:In|My)(?:\s\S+){2}\s(\S+)'
title.str.extract(pattern, expand=False).dropna().head()

### `extract` must have capture groups

The regex used with the `extract` string method must have capture groups. If not, an error will be raised.

### Multiple capture groups for `extract`

You can capture more than one group with `extract`. Take a look at the following regex which captures the first word after a movie that begins with 'The' and the first word after 'of'.

In [ ]:
pattern = r'^The (\S+) of (\S+)'
title.str.extract(pattern).dropna().head()

### Extracting each occurrence

The `extract` method extracts the **first** string matched by the capture group. It's possible to extract each occurrence of the capture group with the `extractall` method. The following pattern matches words that begin with `'C'`. We use `extract` to extract the first word that begins with `'C'` in each movie.

In [ ]:
title.str.extract(r'\b(C\w+)\b').dropna().head()

If we take a look at the movie at index location 13, you'll see that there are two words that begin with `'C'`.

In [ ]:
title.loc[13]

To extract each occurrence use the `extractall` method. It always returns a one-column DataFrame with a multilevel index. The first level is the original index, with the second being the match number as an integer beginning at 0. The movie at index 13 has two words that begin with `'C'` as does the movie at index 16.

In [ ]:
title.str.extractall(r'\b(C\w+)\b').head(6)

In the Cleaning Data part of the book, you'll learn how to reshape this data so that each movie has a single row. There are three columns, because at least one movie has three words beginning with `'C'`.

In [ ]:
title.str.extractall(r'\b(C\w+)\b').squeeze().unstack().head()

## Greedy vs lazy quantifiers

While the `*` and `+` quantifiers were covered in a previous chapter, an important piece of functionality was omitted. By default, each of these quantifiers perform a **greedy** match, continuing to match as many characters that fit the pattern as possible. Instead, they can be made to perform a **lazy** match, by only matching the minimum number of characters that fit the pattern. The greedy vs lazy behavior is easily seen with capture groups, which is why it is introduced in this chapter. Before showing the syntax for a lazy match, let's use the default functionality of the `*` quantifier in an attempt to extract the first word from all movies that begin with `'A'`.

In [ ]:
title.str.extract(r'^(A.*)\b').dropna().head()

Our pattern, `^(A.*)\b`, ends up capturing the entire movie title instead. The issue here is the `.*` component of the pattern which matches any character. The `*` is greedy and continues to consume characters until the very last word boundary is found. The last word boundary is at the end of the string, so the pattern matches the entire title. To make the `*` (or `'+`) lazy, place a `?` after it. A lazy quantifier stops after the very next part of the pattern is matched, which in this case is the first word boundary. Below, we correctly extract the first word of movies beginning with `'A'`.

In [ ]:
title.str.extract(r'^(A.*?)\b').dropna().head()

## Special syntax for parentheses

The parentheses has its own special set of syntax to allow for several different kinds of functionality. This is similar to the square brackets metacharacter, which use the caret (for excluding characters) and hyphen (for separating the start and end of a character class). For parentheses, all extra functionality begins with a **question mark** as the first character, followed by more characters that define the specific functionality. Below, the three dots, `...`, refer to the normal regex within the parentheses.

* `(...)` - capture group
* `(?:...)` - non-capturing group
* `(?P<group_name>...)` - named capturing group. Replace `group_name` with the name of your group.
* `(P=group_name)` - referencing a named capture group
* `(?=...)` - positive lookahead assertion
* `(?!...)` - negative lookahead assertion
* `(?<=...)` - positive lookbehind assertion
* `(?<!...)` - negative lookbehind assertion
* `(?#...)` - comment ignored by the regex engine

The normal grouping and capturing with parentheses (without a question mark inside) has already been covered as well as non-capturing groups. The rest of the chapter will focus on the remainder of the parentheses functionality.

## Referencing previous groups

Each group created within the pattern is given an integer reference beginning at 1. The groups are able to be referenced later in the pattern using `\n`, where n is the integer of the capture group. The below regex creates one group, `(\S{4,})`, matching at least four consecutive non-space characters anchored to the beginning of the string. This group is referenced with `\1` later in the pattern. 

You'll notice that there is a warning since we are not extracting text here. Unfortunately, it isn't possible to reference non-capturing groups, so the warning will remain.

In [ ]:
find_pattern(title, r'^(\S{4,}).*\1').head()

Using a group reference like this does not look for the repeated pattern, but for the exact specific text repeated. The same exact text that began the string must appear elsewhere in the title. Notice that in `'Julie & Julia'`, the first group is `'Juli'` and not `'Julie'`. Likewise with `'Dumb and Dumber To'`, the group is `'Dumb'`. It is possible to add word boundaries to match whole words.

In [ ]:
find_pattern(title, r'^(\S{4,})\b.*\b\1\b').head()

### Naming groups

Referencing groups with integers is easy enough when there are just a few, but can become difficult to keep track of if there are many of them (some complex regexes have dozens of groups). To explicitly name a group, place `?P<group_name>` immediately after the parentheses. Reference it later in the pattern with `(?P=group_name)`. You are free to name the group using the same rules that apply for Python variable names. Here, we recreate our pattern above with a named capturing group.

In [ ]:
find_pattern(title, r'^(?P<first_word>\S{4,})\b.*\b(?P=first_word)\b').head()

### Extracting named groups

When using named groups with `extract` the resulting columns will be given the name of the group. This is much nicer than the integers pandas uses by default. Below, we extract the first word of the movie and optionally the last word - the last question mark makes the entire last word optional. This results in a DataFrame with descriptive column names.

In [ ]:
title.str.extract(r'^(?P<first_word>\S+)\b.*?\b(?P<last_word>\S+)?$').head()

## Lookaheads and lookbehinds

Lookaheads and lookbehinds are a class of functionality also referred to as "lookarounds". They do precisely what their names entail, and look ahead of or behind the current position for a match. The lookarounds do not consume any characters (they are zero-width) and merely test whether or not the pattern is matched ahead or behind the current position. Summarizing in a single sentence - They are a boolean test to determine if a pattern matches ahead or behind the current position. There are four types of lookarounds:

* **Positive lookahead** - `(?=...)` - evaluates as true when pattern is matched ahead of current position
* **Negative lookahead** - `(?!...)` - evaluates as true when pattern is NOT matched ahead of current position 
* **Positive lookbehind** - `(?<=...)` - evaluates as true when pattern is matched behind current position 
* **Negative lookbehind** - `(?<!...)` - evaluates as true when pattern is NOT matched behind current position 

## Positive lookahead assertion - `(?=...)`

The positive lookahead assertion looks ahead from the current position to match the given regex. The word "assertion" here is important and refers to it asserting (only determining whether it is true or false), but not consuming any characters. Notice in the example below that there is no warning emitted by pandas. All lookaround assertions do NOT capture, just group the regex that it contains and asserts whether or not a match is found. Let's find all movie titles that have the word `'the'` in them followed by the word `'man'` (case insensitive) somewhere else in the title with the regex `the(?=man)`. 

In [ ]:
find_pattern(title, r'the(?=man)', flags=re.I)

It fails to find any matches, but this is because the lookahead attempts to match the pattern immediately at the current position, which is the character following `'e'` in `'the'`. Since `'theman'` is not found in any titles, none are returned. To look for the word `'man'` following `'the'` somewhere in the string, we'll have to precede it by `'.*'` which will match any number of characters until `'man'` is found.

In [ ]:
find_pattern(title, r'the(?=.*man)', flags=re.I).head()

In this particular example, the positive lookahead is not necessary, as a regex without it is able to do the same task. We verify that the number of matches is the same for each one below. It's a good idea to use simpler regex when multiple options exist to complete the same task.

In [ ]:
find_pattern(title, r'the(?=.*man)', flags=re.I).size

In [ ]:
find_pattern(title, r'the.*man', flags=re.I).size

## Logical AND operator with positive lookaheads

A necessary use for a positive lookahead is to create a logical **AND** operation. We previously saw that the pipe and square brackets metacharacters allow for different OR operations, but there is no single special character for AND operations. Let's say we are interested in finding movies that have both `'the'` and `'man'` somewhere in the title with order not mattering. This type of pattern is only possible with positive lookaheads.

In [ ]:
find_pattern(title, r'^(?=.*the)(?=.*man)', flags=re.I).head(6)

The pattern uses two consecutive positive lookaheads both beginning with `.*`, allowing for any number of characters to be matched before the desired words. In this example, the pattern is anchored to the start of the string. The current position is the very first position of the string, so each of the positive lookaheads start with the first character and look for their respective word until the end of the string. 

The same result will be produced if the pattern is not anchored to the start, but will take much more computational time. The regex engine will use every single character as the current position if not anchored to the start, repeating work unnecessarily. Most positive lookaheads will be anchored to the start of the string.

Let's change our search to find titles that have both `'the`' and `'man'` in them, but only after the first word. Now, the current position of the lookahead is the space following the first word. We also add word boundaries to look for the exact words.

In [ ]:
find_pattern(title, r'^\w+ (?=.*\bthe\b)(?=.*\bman\b)', flags=re.I)

Let's say we are interested in extracting the second word in each of the titles above. Because lookaheads do not advance the current position, we simply place a capture group for a single word at the end of the regex.

In [ ]:
pattern = r'^\w+ (?=.*\bthe\b)(?=.*\bman\b)(\w+)'
title.str.extract(pattern, flags=re.I).dropna()

Any number of positive lookaheads can be used consecutively to create multiple AND conditions. Here we test for titles that contain `'the'`, `'of'`, and `'an'`.

In [ ]:
find_pattern(title, r'^(?=.*\bthe\b)(?=.*\bof\b)(?=.*\ban\b)', flags=re.I)

## Negative lookahead assertion - `(?!...)`

As the name implies, a negative lookahead assertion tests for a pattern that does not match ahead of the current position. The following negative lookahead finds titles that contain `'Star'` but are not immediately followed by `' Wars'` or `' Trek'`.

In [ ]:
find_pattern(title, r'Star(?! Wars| Trek)').head()

Negative lookaheads enable us to find all movies that do not contain a particular word. Take a look at the following regex, which appears to exclude movies containing the word `'the'`.

In [ ]:
find_pattern(title, r'(?!.*the)', flags=re.I).head()

This regex matches every single string because the engine checks for the pattern at every single position. Since there can't be any matches at the end of the string, this negative lookahead always evaluates as true. We must anchor it to the start of the string for it to use a single current position.

In [ ]:
find_pattern(title, r'^(?!.*the)', flags=re.I).head()

## Positive lookbehind assertion - `(?<=...)`

Lookbehind assertions are relatively new features to Python that look behind the current position to test for a particular pattern. Lookbehinds are not as flexible as lookaheads because the pattern must be **fixed-width**. This means that the `*`, `+`, and `?` special characters cannot be used as well as any pattern where the number of characters matched could change. Let's use a positive lookbehind to match titles that have a lower case letter ranging between `'m'` and `'t'` preceding a space and two digits.

In [ ]:
find_pattern(title, r'(?<=[m-t]) \d{2}').head()

Positive lookaheads are rarely necessary, as the above pattern can be reproduced using `[m-t] \d{2}`.

## Negative lookbehind assertion - `(?<!...)`

Negative lookbehind assertions are far more useful than their positive counterparts. Let's say we are interested in finding all movies that have the word `'Wars'` in them but don't want to include `'Star Wars'` movies. It's necessary to use a negative lookbehind to test for the non-appearance of the `'Stars '` before `'Wars'`.

In [ ]:
find_pattern(title, r'(?<!Star )Wars')

Notice that the third movie contains `'Star Wars'` but still matches the pattern because of `'Clone Wars'`. The following regex uses both negative and positive *lookaheads* to exclude `'Star Wars'`, but keep movies that contain `'Wars'`. 

In [ ]:
find_pattern(title, r'^(?!.*Star Wars)(?=.*Wars)')

## Exercises

### Exercise 1

<span style="color:green; font-size:16px">Find all movies that begin with the exact words `'I'`, `'An'`, or `'How'`, are followed by at least 20 characters, and end in lowercase `'n'` through '`s'`. Make sure there is no warning.</span>

### Exercise 2

<span style="color:green; font-size:16px">Find all movies that have a lower case consonant followed by a lowercase vowel followed immediately by a repeat of those same two letters. For example, `'Banana'` repeats `'na'` twice in succession.</span>

### Exercise 3

<span style="color:green; font-size:16px">For all movies that begin with 'The' and are followed by the next word that begins with a digit, extract just the digits part of the word.</span>

### Exercise 4

<span style="color:green; font-size:16px">Find all movies that have two separate numbers in them. An example would be, '7 days and 7 nights'.</span>

### Exercise 5

<span style="color:green; font-size:16px">Find all movies that have six or more non-vowel and non-space characters in a row.</span>

### Exercise 6

<span style="color:green; font-size:16px">Extract the very next non-space character after 't' or 'T' for each movie, convert to lowercase, and then return the count of each character.</span>

### Exercise 7

<span style="color:green; font-size:16px">Extract all the words that begin with 'T' or 't' and end in 'e' then find their frequency converting to lowercase first.</span>

### Exercise 8

<span style="color:green; font-size:16px">Find all movies containing a 6, 7, and 8 character word.</span>

### Exercise 9

<span style="color:green; font-size:16px">Find all movies that have a `'q'` that is not followed by a `'u'`.</span>

### Exercise 10

<span style="color:green; font-size:16px">Find all movies containing an `'s'` that is not preceded by a vowel or a single quote mark. The `'s'` cannot be the first or last letter of the word. Ignore case.</span>

### Exercise 11

<span style="color:green; font-size:16px">Find all movies where the first four characters of the first word repeat at least 10 characters after the fourth character somewhere else in the title. Use a named group.</span>

### Exercise 12

<span style="color:green; font-size:16px">Find all movies that repeat one character three or more times in a row.</span>

### Exercise 13

<span style="color:green; font-size:16px">Find all movies where a word of at least four characters in length repeats but has a different starting letter. For example, `'Hocus Pocus'` and `'Kill Bill'` both work.</span>

### Exercise 14

<span style="color:green; font-size:16px">Extract all characters from the first digit through the last. Output the first 10 non-missing rows.</span>

### Exercise 15

<span style="color:green; font-size:16px">Extract all characters from the first digit through the second digit. Output the first 10 non-missing rows.</span>

### Exercise 16

<span style="color:green; font-size:16px">Extract all characters from the first word that ends in `'e'` through the next word that ends in `'e'`. Ignore case.</span>